## Import packages

In [1]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import datetime
import pandas as pd
import numpy as np
import csv
import re
from textblob import TextBlob
import schedule
import string
import preprocessor as p
import os
import time
import nltk
import matplotlib
from twitter_scrape import TweetMiner
from datetime import date
import warnings
import requests

warnings.filterwarnings('ignore')

## Tweets collection

In [31]:
miner = TweetMiner(result_limit = 500) #Mine tweet. 
uk_tweets = miner.mine_user_tweets(user='DHSCgovUK', max_pages=10)

In [26]:
def get_tweets_dataframe(mined_tweets):
    """Take mined tweets as args, return a tweets dataframe."""
    uk_tweets_df= pd.DataFrame(mined_tweets)
    update_df = mined_tweets_df[mined_tweets_df['text'].str.contains("died")] # Only include coronavirus update  
    update_df = update_df[['screen_name', 'created_at', 'text']].reset_index(drop=True) # Keep relevant columns 
    update_df['date'] = virus_df.created_at.dt.strftime('%Y-%m-%d') # Get date 
    update_df['time'] = virus_df.created_at.dt.strftime('%H:%M:%S') # Get time 
    return update_df
# uk_df = get_tweets_df(uk_tweets)

,screen_name,created_at,text,date,time
0,DHSCgovuk,2020-04-25 14:19:34,"As of 9am 25 April, 640,792 tests have conclud...",2020-04-25,14:19:34
1,DHSCgovuk,2020-04-24 14:13:49,"As of 9am 24 April, 612,031 tests have conclud...",2020-04-24,14:13:49
2,DHSCgovuk,2020-04-23 13:56:11,"As of 9am 23 April, 583,496 tests have conclud...",2020-04-23,13:56:11
3,DHSCgovuk,2020-04-22 13:47:57,"As of 9am 22 April, 559,935 tests have conclud...",2020-04-22,13:47:57
4,DHSCgovuk,2020-04-21 15:08:03,"As of 9am 21 April, 535,342 tests have conclud...",2020-04-21,15:08:03
5,DHSCgovuk,2020-04-20 15:26:27,"As of 9am 20 April, 501,379 tests have conclud...",2020-04-20,15:26:27
6,DHSCgovuk,2020-04-19 13:14:32,"As of 9am 19 April, 482,063 tests have conclud...",2020-04-19,13:14:32
7,DHSCgovuk,2020-04-18 13:04:37,"As of 9am 18 April, 460,437 tests have conclud...",2020-04-18,13:04:37
8,DHSCgovuk,2020-04-17 13:06:33,"As of 9am 17 April, 438,991 tests have conclud...",2020-04-17,13:06:33
9,DHSCgovuk,2020-04-16 13:44:13,"As of 9am 16 April, 417,649 tests have conclud...",2020-04-16,13:44:13


## Data cleaning

In [58]:
update_df = virus_df.loc[(virus_df['date'] > '2020-04-05')] # 
update_df['test_cum'] = update_df.text.str.findall('\d+(?:,\d+)?').str[2] # Toral number 
update_df['test_daily'] = update_df.text.str.findall('\d+(?:,\d+)?').str[3]
update_df['ppl_test_cum'] = update_df.text.str.findall('\d+(?:,\d+)?').str[5]
update_df['ppl_confirmed_cum'] = update_df.text.str.findall('\d+(?:,\d+)?').str[6]
update_df['death_cum'] = update_df.text.str.findall('\d+(?:,\d+)?').str[9]
update_df['url'] = update_df.text.str.findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+').apply(''.join)

,screen_name,created_at,text,date,time,url,test_cum,test_daily,ppl_test_cum,ppl_confirmed_cum,death_cum
0,DHSCgovuk,2020-04-25 14:19:34,"As of 9am 25 April, 640,792 tests have conclud...",2020-04-25,14:19:34,https://t.co/5HLhOFWdlu,"640,792","28,760","517,836","148,377","20,319"
1,DHSCgovuk,2020-04-24 14:13:49,"As of 9am 24 April, 612,031 tests have conclud...",2020-04-24,14:13:49,https://t.co/ixQBaugnGh,"612,031","28,532","444,222","143,464","19,506"
2,DHSCgovuk,2020-04-23 13:56:11,"As of 9am 23 April, 583,496 tests have conclud...",2020-04-23,13:56:11,https://t.co/0quyQMCheo,"583,496","23,560","425,821","138,078","18,738"
3,DHSCgovuk,2020-04-22 13:47:57,"As of 9am 22 April, 559,935 tests have conclud...",2020-04-22,13:47:57,https://t.co/gFFpwZe1gl,"559,935","22,814","411,192","133,495","18,100"
4,DHSCgovuk,2020-04-21 15:08:03,"As of 9am 21 April, 535,342 tests have conclud...",2020-04-21,15:08:03,https://t.co/rLnm7MWxEw,"535,342","18,206","397,670","129,044","17,337"
5,DHSCgovuk,2020-04-20 15:26:27,"As of 9am 20 April, 501,379 tests have conclud...",2020-04-20,15:26:27,https://t.co/c5suKHMEIS,"501,379","19,316","386,044","124,743","16,509"
6,DHSCgovuk,2020-04-19 13:14:32,"As of 9am 19 April, 482,063 tests have conclud...",2020-04-19,13:14:32,https://t.co/sO9IRInBsu,"482,063","21,626","372,967","120,067","16,060"
7,DHSCgovuk,2020-04-18 13:04:37,"As of 9am 18 April, 460,437 tests have conclud...",2020-04-18,13:04:37,https://t.co/yZmas1wSvS,"460,437","21,389","357,023","114,217","15,464"
8,DHSCgovuk,2020-04-17 13:06:33,"As of 9am 17 April, 438,991 tests have conclud...",2020-04-17,13:06:33,https://t.co/BkOC0O9EUy,"438,991","21,328","341,551","108,692","14,576"
9,DHSCgovuk,2020-04-16 13:44:13,"As of 9am 16 April, 417,649 tests have conclud...",2020-04-16,13:44:13,https://t.co/z6FNsommq9,"417,649","18,665","327,608","103,093","13,729"


In [42]:
vis_df = update_df.drop(columns=['text', 'created_at', 'time'])
# vis_df= vis_df.stack().str.replace(',', '').unstack()
# vis_df = vis_df.apply(pd.to_numeric, errors='ignore')
# vis_df['ppl_tested_daily'] = vis_df.ppl_test_cum.diff(periods = -1).fillna(0).astype(np.int64)
# vis_df['death_daily'] = vis_df.death_cum.diff(periods=-1).fillna(0).astype(np.int64)
# vis_df['ppl_confirmed_daily'] = vis_df.ppl_confirmed_cum.diff(periods = -1).fillna(0).astype(np.int64)
# vis_df['ppl_percentage_daily'] = vis_df['ppl_confirmed_daily']/vis_df['ppl_tested_daily']
# vis_df['death_rate'] = vis_df['death_cum']/vis_df['ppl_confirmed_cum']
# vis_df['ppl_confirmed_change'] = vis_df.ppl_confirmed_daily.pct_change(periods=-1)
# vis_df['death_change'] = vis_df.death_daily.pct_change(periods=-1)
# vis_df['ppl_tested_change'] = vis_df.ppl_tested_daily.pct_change(periods=-1)
vis_df

,screen_name,date,url,test_cum,test_daily,ppl_test_cum,ppl_confirmed_cum,death_cum
0,DHSCgovuk,2020-04-25,"[(https, t.co, /5HLhOFWdlu)]","640,792","28,760","517,836","148,377","20,319"
1,DHSCgovuk,2020-04-24,"[(https, t.co, /ixQBaugnGh)]","612,031","28,532","444,222","143,464","19,506"
2,DHSCgovuk,2020-04-23,"[(https, t.co, /0quyQMCheo)]","583,496","23,560","425,821","138,078","18,738"
3,DHSCgovuk,2020-04-22,"[(https, t.co, /gFFpwZe1gl)]","559,935","22,814","411,192","133,495","18,100"
4,DHSCgovuk,2020-04-21,"[(https, t.co, /rLnm7MWxEw)]","535,342","18,206","397,670","129,044","17,337"
5,DHSCgovuk,2020-04-20,"[(https, t.co, /c5suKHMEIS)]","501,379","19,316","386,044","124,743","16,509"
6,DHSCgovuk,2020-04-19,"[(https, t.co, /sO9IRInBsu)]","482,063","21,626","372,967","120,067","16,060"
7,DHSCgovuk,2020-04-18,"[(https, t.co, /yZmas1wSvS)]","460,437","21,389","357,023","114,217","15,464"
8,DHSCgovuk,2020-04-17,"[(https, t.co, /BkOC0O9EUy)]","438,991","21,328","341,551","108,692","14,576"
9,DHSCgovuk,2020-04-16,"[(https, t.co, /z6FNsommq9)]","417,649","18,665","327,608","103,093","13,729"


In [6]:
latest_date = vis_df.date.iloc[0]
yesterday = vis_df.date.iloc[1]
ppl_test_cum = format(vis_df.ppl_test_cum.iloc[0], ',')
ppl_confirmed_daily = format(vis_df.ppl_confirmed_daily.iloc[0], ',')
ppl_tested_daily = format(vis_df.ppl_tested_daily.iloc[0], ',')
yesterday_cases = format(vis_df.ppl_confirmed_daily.iloc[1], ',')
ppl_confirmed_change = round(vis_df.ppl_confirmed_change.iloc[0]*100, 1)
ppl_tested_change = round(vis_df.ppl_tested_change.iloc[0]*100, 1)
death_daily = format(vis_df.death_daily.iloc[0], ',')
death_rate = round(vis_df.death_rate.iloc[0]*100, 1)
death_cum = format(vis_df.death_cum.iloc[0], ',')
death_change = round(vis_df.death_change.iloc[0]*100, 1)

In [7]:
a = 'On {0}, {1} people are tested, a {2}% change.'.format(latest_date, ppl_tested_daily, ppl_tested_change)
b = ' So far, {0} people have been tested of which {1} tested positive today.'.format(ppl_test_cum, ppl_confirmed_daily)
c = ' There is a {0}% change in confirmed cases, comparing to {1} cases yesterday.'.format(ppl_confirmed_change, yesterday_cases)
d = ' Death toll increased by {0} to {1}, representing a {2}% change. The death rate is {3}%.'.format(death_daily, death_cum, death_change,death_rate)
e = a+b+c+d
print (e)

On 2020-04-25, 73,614 people are tested, a 300.1% change. So far, 517,836 people have been tested of which 4,913 tested positive today. There is a -8.8% change in confirmed cases, comparing to 5,386 cases yesterday. Death toll increased by 813 to 20,319, representing a 5.9% change. The death rate is 13.7%.


In [8]:
def telegram_bot_sendtext(bot_message):
        bot_token = '1157266597:AAEAEJN67IFaAHuCpl43dqCg_GlBq2xKPyo'
        bot_chatID = '@ukcoronavirusupdate'
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
        response = requests.get(send_text)


In [9]:
output_df = pd.read_csv('/Users/linusnhh/Desktop/local/Python/twitter/project_tweet/output/uk_coronavirus_stats.csv')
if (date.today().strftime('%Y-%m-%d') == latest_date) == False:
    print ('Official data has not been updated yet')
if (date.today().strftime('%Y-%m-%d') != output_df.date.iloc[0]) == True: 
    print ('Latest data has already been written.')
if date.today().strftime('%Y-%m-%d') == latest_date and date.today().strftime('%Y-%m-%d') != output_df.date.iloc[0]:
    print('Sending message...')
    telegram_bot_sendtext(e)
    vis_df.to_csv('/Users/linusnhh/Desktop/local/Python/twitter/project_tweet/output/uk_coronavirus_stats.csv', index = False)
    print('Data has been updated.')